In [ ]:
# 토큰 정보로드를 위한 라이브러리
# 설치: pip install python-dotenv
from dotenv import load_dotenv

# 토큰 정보로드
load_dotenv()

## 🌱 DALL-E

- 웹사이트: [DALL-E](https://labs.openai.com/)
- 웹사이트에서 이미지를 업로드 하거나, 이미지를 생성하는 기능 구현이 잘 되어 있습니다.
- in-painting, out-painting 도구도 함께 제공합니다.


## Client 생성

- `client` 는 OpenAI 모듈로 생성된 인스턴스 입니다.

[주의] 아래의 코드에서 오류가 난다면 API 키의 오류일 가능성이 높습니다.


설치된 openai 버전 확인


In [ ]:
import openai

print(f"설치된 버전: {openai.__version__}")

In [ ]:
from openai import OpenAI

client = OpenAI()

## ① 이미지 생성

- Reference: https://platform.openai.com/docs/guides/images/introduction?context=node
- `model`
  - 생성: 사용 가능한 모델은 `dall-e-3` 및 `dall-e-2`
  - 편집(edit): `dall-e-2`
  - 기준 이미지에 대한 다양화(variation): `dall-e-2`

**이미지 크기**

- `dall-e-2`
  - `256x256`, `512x512`, `1024x1024`
- `dall-e-3`
  - `1024x1024`, `1792x1024`, `1024x1792`

기본적으로 이미지는 표준 화질로 생성되지만, DALL-E 3을 사용할 때는 화질을 설정할 수 있습니다. "HD"로 설정하면 디테일이 향상됩니다. 정사각형의 표준 화질 이미지가 가장 빠르게 생성됩니다.

**사진요청 개수**

DALL-E 3을 사용하면 **한 번에 1개의 이미지를 요청** 할 수 있고(병렬 요청을 통해 더 많은 이미지를 요청할 수 있음), DALL-E 2를 사용하면 n 매개변수와 함께 **한 번에 최대 10개의 이미지를 요청** 할 수 있습니다.


- 참고: https://dallery.gallery/the-dalle-2-prompt-book/


In [ ]:
# DALL-E 3 모델을 사용하여 이미지를 생성합니다.
response = client.images.generate(
    model="dall-e-3",
    prompt="A detailed neoclassicism painting depicting the frustration of being put on hold during a phone call(iphone)",
    size="1024x1024",
    quality="standard",
    n=1,
)

# 생성된 이미지의 URL을 저장합니다.
image_url = response.data[0].url

다운로드 받을 수 있는 이미지의 URL을 출력합니다.


In [ ]:
# 생성된 이미지의 URL을 출력합니다.
print(image_url)

생성된 결과 (예시)

<style>
.container {
  display: flex; /* Flexbox를 사용하여 요소들을 나란히 배치 */
}

.img-column {
  flex: 1; /* 컨테이너 내의 공간을 동일하게 분배 */
  padding: 10px; /* 이미지 사이에 약간의 공간을 두기 위한 패딩 */
}

.img-column img {
  width: 100%; /* 이미지를 부모 요소의 크기에 맞춤 */
  height: auto; /* 이미지의 비율을 유지 */
}
</style>

<div class="container">
  <div class="img-column">
    <img src="data/generated_image1.jpg" alt="Generated Image 1">
  </div>
  <div class="img-column">
    <img src="data/generated_image2.jpg" alt="Generated Image 2">
  </div>
</div>


Jupyter Notebook 에서 출력하기 위하여 Image 를 사용합니다.


In [ ]:
from IPython.display import Image

# 생성된 이미지를 출력합니다.
Image(url=image_url, width=500)

`urllib` 라이브러리를 사용하여 이미지를 다운로드합니다.


In [ ]:
import urllib

# 생성된 이미지를 URL로부터 다운로드하여 저장합니다.
urllib.request.urlretrieve(image_url, "generated_image.jpg")

### ✔️ DALLE-3 프롬프트 예시

- 1️⃣ 예시 프롬프트:
  - `a closeup of a female face with headphones in retro colors, synthwave style, 2d digital vector art`
- 2️⃣ 예시 프롬프트:

  - `A detailed neoclassicism painting depicting the frustration of being put on hold during a phone call(iphone)`

- 더 많은 예시는 다음의 링크를 참고해 보세요:
  - https://dallery.gallery/the-dalle-2-prompt-book/


In [ ]:
response = client.images.generate(
    model="dall-e-3",
    prompt="A sunlit indoor lounge area with a pool with clear water"
    "and another pool with translucent pastel pink water, next"
    " to a big window, digital art",
    size="1024x1024",
    quality="standard",
    n=1,
)

image_url = response.data[0].url

생성된 이미지를 출력합니다.


In [ ]:
print(image_url)

In [ ]:
# 생성된 이미지를 출력합니다.
Image(url=image_url, width=500)

In [ ]:
# 생성된 이미지를 URL로부터 다운로드하여 저장합니다.
urllib.request.urlretrieve(image_url, "generated_image2.jpg")

## ② 이미지 수정(Image Edit)

- 도큐먼트: https://platform.openai.com/docs/api-reference/images/createEdit

**주요 파라미터**

- `mask`:
  - 마스킹 영역은 완전히 투명한 영역(예: 알파가 0인 곳)이 이미지를 편집할 위치를 나타내는 추가 이미지입니다.
  - 4MB 미만의 유효한 PNG 파일이어야 하며 이미지와 크기가 같아야 합니다.
  - 마스킹한 영역에 대하여 다양한 버전의 이미지를 생성합니다.
- `model`: 이미지 생성에 사용할 모델입니다. 현재 `dall-e-2` 만 지원됩니다.
- `n`: 생성할 이미지의 개수입니다.
- `size`: `256x256`, `512x512`, or `1024x1024`. 기본값은 `1024x1024` 입니다.


In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.images.edit(
    model="dall-e-2",
    image=open("data/sample.png", "rb"),
    mask=open("data/sample-mask.png", "rb"),
    prompt="add a Christmas tree",
    n=1,
    size="1024x1024",
)

image_url = response.data[0].url

이전의 코드에서 작성한 프롬프트는 마치 사진과 같은 Photorealistic 이미지를 생성하기 위하여 카메라 렌즈 정보를 추가해 주었습니다. 이렇게 카메라 렌즈, 화각, 조리개 값등을 넣어 마치 사진으로 촬영한 듯한 이미지를 생성할 수 있습니다.


원본 이미지는 다음과 같습니다.


In [ ]:
# 원본 이미지를 출력합니다.
Image(url="data/sample.png")

다음은 마스킹한 이미지입니다. 마스킹은 photopea 에서 쉽게 진행할 수 있습니다.

- [photopea 링크](https://photopea.com/)


In [ ]:
# 마스킹된 이미지를 출력합니다.
Image(url="data/sample-mask.png")

마지막으로 마스킹된 영역을 크리스마스 트리로 채운 결과 이미지 입니다.


In [ ]:
# 생성된 이미지를 URL로부터 다운로드하여 저장합니다.
urllib.request.urlretrieve(image_url, "edited_output.png")

# 생성된 이미지를 출력합니다.
Image(url="edited_output.png")

## ③ 다양한 버전의 이미지 생성(Image Variation)

주어진 이미지의 변형을 생성합니다.

- 도큐먼트: https://platform.openai.com/docs/api-reference/images/createVariation

주요 파라미터

- `image`: 변형의 기준으로 사용할 이미지입니다. 4MB 미만의 유효한 `PNG` 파일이어야 하며 `정사각형` 이어야 합니다.
- `model`: 이미지 생성에 사용할 모델입니다. 현재 `dall-e-2` 만 지원됩니다.
- `n`: 생성할 이미지의 개수입니다.
- `size`: `256x256`, `512x512`, or `1024x1024`. 기본값은 `1024x1024` 입니다.


In [ ]:
number_of_variations = 2
image_size = "512x512"

response = client.images.create_variation(
    image=open("data/generated_image_porche.png", "rb"),  # 기준 이미지
    n=number_of_variations,  # 생성할 이미지의 개수
    size=image_size,  # 생성할 이미지의 크기
)

image_url = response.data[0].url

생성된 이미지를 출력합니다.


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.image import imread


fig, ax = plt.subplots(1, number_of_variations,
                       figsize=(8, 8 * number_of_variations))

for i in range(1, number_of_variations + 1):
    filename = f"variated_image_{i}.jpg"
    i -= 1
    urllib.request.urlretrieve(response.data[i].url, filename)
    img = imread(filename)
    ax[i].imshow(img)
    ax[i].axis("off")
    ax[i].set_title(f"Variation {i+1}")

plt.tight_layout()
plt.show()